In [1]:
import os
from slgep_lib import wrap_config
from utils import Saver
from cea import cea
import argparse
import yaml
from tools import Tools
from atari_benchmark import *
from slgep_lib import *


In [2]:
# Load configuration
config = yaml.load(open('config.yaml').read())

# Load benchmark
singletask_benchmark = yaml.load(open('atari_benchmark/singletask-benchmark.yaml').read())
data = singletask_benchmark['single-2']
config.update(data)
config = wrap_config(config)


In [3]:
from joblib import Parallel, delayed
from os import cpu_count
from tqdm import trange


taskset = Taskset(config)
# Model
cf = ChromosomeFactory(config)
# Simple parameter
K = taskset.K
config['K'] = K
N = config['pop_size'] * K
T = config['num_iter']
mutation_rate = config['mutation_rate']
# Initialization
population = cf.initialize()
print(cf.chromosome_range)


ChromosomeRange(R1=6, R2=14, R3=16, R4=144)


In [4]:
permutation_index = np.random.permutation(10)
print(permutation_index)

[7 9 6 5 3 0 2 4 8 1]


In [5]:
population[:10] = population[:10][permutation_index]
print(population)
print(population.shape)

[[ 4 12  8 ... 15 14 14]
 [ 6  8  9 ... 14 15 15]
 [ 4 13  0 ... 14 14 15]
 ...
 [ 4 13  8 ... 15 15 14]
 [ 5  8  8 ... 14 15 14]
 [ 4 12  7 ... 14 15 15]]
(20, 108)


In [ ]:
low, hight = cf._get_feasible_range(10)
x = population.T[10]
print(x)
print(low, hight)
y = np.bincount(x, minlength = hight)
y = y[low:]
print(y.shape)

## Tính xác suất trung bình cho từng chiều trong dims

In [9]:
for i in range(population.shape[1]):
    low, hight = cf._get_feasible_range(i)
    print(low,hight)
    prob_arr = np.bincount(population.T[i], minlength = hight)
    print(prob_arr.shape)
    prob_arr = prob_arr[low:]/population.shape[0]
    print('dim',i)
    print(prob_arr)

0 14
(14,)
gen 0
[0.1  0.1  0.1  0.   0.2  0.05 0.05 0.05 0.05 0.05 0.05 0.1  0.   0.1 ]
0 14
(14,)
gen 1
[0.   0.   0.05 0.15 0.1  0.   0.   0.1  0.15 0.   0.05 0.1  0.15 0.15]
0 14
(14,)
gen 2
[0.05 0.   0.   0.   0.15 0.1  0.1  0.1  0.15 0.1  0.   0.15 0.   0.1 ]
0 14
(14,)
gen 3
[0.   0.05 0.1  0.2  0.1  0.05 0.05 0.15 0.   0.05 0.   0.15 0.1  0.  ]
0 14
(14,)
gen 4
[0.   0.   0.1  0.1  0.05 0.   0.05 0.15 0.1  0.1  0.05 0.1  0.15 0.05]
0 14
(14,)
gen 5
[0.   0.05 0.1  0.   0.2  0.15 0.15 0.05 0.   0.1  0.05 0.   0.05 0.1 ]
16 144
(144,)
gen 6
[0.05 0.1  0.05 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.05 0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.05 0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.05 0.   0.   0.05 0.   0.   0.   0.   0.05 0.
 0.   0.05 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.05 0.05
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.05 0.05 0.
 0.   0.   0.   0.05 0.   0.  

## Xác suất trong 1 chiều + sample 

In [15]:
i = 2
low, hight = cf._get_feasible_range(i)
prob_arr = np.bincount(population.T[i], minlength = hight)
prob_arr = prob_arr[low:]/population.shape[0]
list = np.arange(low, hight)
print(list)
print(prob_arr)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
[0.05 0.   0.   0.   0.15 0.1  0.1  0.1  0.15 0.1  0.   0.15 0.   0.1 ]


In [30]:
y = np.random.choice(list, population.shape[0], p = prob_arr.tolist())
print(y)

[ 8  6  0  8  9  4 11  9  6  9  8 11  7  8  8  0 13 13  7  8]
